# **Second PML Project**
Ghadamiyan Lida, class 407 AI

### Libraries

In [ ]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
nltk.download('wordnet')
nltk.download('punkt')
stemmer = SnowballStemmer('russian')

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler

from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans, DBSCAN

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import fowlkes_mallows_score

import seaborn as sns

from yellowbrick.text import FreqDistVisualizer, TSNEVisualizer

from sklearn.pipeline import Pipeline 
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV

## Data Preprocessing

In [ ]:
data = pd.read_csv('../input/russian-language-toxic-comments/labeled.csv')

In [ ]:
# Plotting the distribution of the labels
toxicc = 0
okc = 0
for label in data['toxic']:
    if label == 1:
        toxicc += 1
    if label == 0:
        okc += 1

In [ ]:
plt.bar('0 - Non toxic comments', okc)
plt.bar('1 - Toxic comments', toxicc, color = 'orange')
plt.show()

In [ ]:
data2 = []
for i in range(0, len(data.index)):

    # Punctuation removal
    table = str.maketrans(dict.fromkeys(string.punctuation))                   
    sentences = (data.comment[i].translate(table))

    # " '\n " removal
    words = sentences[:-3] 

    # Tokenization
    words = nltk.word_tokenize(words)

    # shrt words removal & lemmatization & stemming
    words_ = []
    for word in words:
        if len(word) > 2:   
            if not word.isnumeric():                                                  
                word1 = stemmer.stem(WordNetLemmatizer().lemmatize(word, pos='v'))          
                words_.append(word1)
    data2.append(words_)

df = pd.DataFrame({'comment':data2, 'toxic':data['toxic']})   

In [ ]:
comparison = pd.DataFrame({'comments': data['comment'], 'preprocessed comments': data2, 'labels': data['toxic']})
comparison.head()

In [ ]:
train_data__, test_data__, train_labels, test_labels = train_test_split(df['comment'], df['toxic'], test_size = 0.2, random_state = 25)

In [ ]:
# CountVectorizer & TermFrequencies
cvect = CountVectorizer(ngram_range=(1, 1), lowercase='true')   
tfidf_transformer = TfidfTransformer(norm= 'l2', use_idf= False)

In [ ]:
# Transforming the processed data to a list (for tfidf)
data4 = train_data__.astype(str).values.tolist()

train_data1 = cvect.fit_transform(data4)
train_data = tfidf_transformer.fit_transform(train_data1)

In [ ]:
# Same procedure for the test data
data5 = test_data__.astype(str).values.tolist()

test_data1 = cvect.transform(data5)
test_data = tfidf_transformer.transform(test_data1)

In [ ]:
# Same procedure for the entire data set
train_data6 = df['comment'].astype(str).values.tolist()

data1_ = cvect.fit_transform(train_data6)
data_ = tfidf_transformer.fit_transform(data1_)

### Data visualization

In [ ]:
#https://www.scikit-yb.org/en/latest/api/text/freqdist.html
features = cvect.get_feature_names()

visualizer = FreqDistVisualizer(features=features)
visualizer.fit(data1_)
visualizer.poof()

In [ ]:
#https://www.scikit-yb.org/en/latest/api/text/freqdist.html

tsne = TSNEVisualizer()
tsne.fit_transform(data_, df['toxic'])
tsne.poof()

## Supervised learning method - Naive Bayes

In [ ]:
# Model fitting
model = MultinomialNB(alpha = 0.1)
model.fit(train_data, train_labels)

In [ ]:
# Prediction
prediction = model.predict(test_data)

We are using two scores to compare the results

Accuracy score = $\frac{TP + TN}{Total}$

Fowlkes Mallwows Score = $\frac{TP}{\sqrt(TP+FP)(TP+FN}$ 

In [ ]:
accuracy_score(test_labels, prediction) 

In [ ]:
print(fowlkes_mallows_score(prediction, test_labels))

In [ ]:
models = Pipeline([('CountVect', CountVectorizer()), 
                     ('TermFreq', TfidfTransformer()), 
                     ('NB', MultinomialNB())]) 

parameters = { 'CountVect__ngram_range': [(1, 1), (1, 2), (2, 2),(4,5)], 
              'TermFreq__use_idf': (True, False), 
              'TermFreq__norm': ('l1', 'l2'), 
              'NB__alpha': [1, 1e-1, 1e-2, 1e-3] } 

CrossValFolds = 5
grid_search= GridSearchCV(models, parameters, cv = CrossValFolds, n_jobs = -1) 
grid_search.fit(data4, train_labels)

print(grid_search.best_score_) 
print(grid_search.best_params_)

In [ ]:
print(classification_report(test_labels, prediction))

In [ ]:
confusion_matrix(test_labels, prediction, )

In [ ]:
ax = sns.heatmap(confusion_matrix(test_labels, prediction), annot = np.array([['1857', '78'],['350', '598']]), cmap=plt.cm.Blues, fmt = '')

# Unsupervised methods

## K-means

In [ ]:
kmeans = KMeans(n_clusters=2, init='k-means++', random_state=0).fit(data_)

In [ ]:
kmeans_pred = []
for label in kmeans.predict(data_):
    if label == 0:
        kmeans_pred.append(1)
    else:
        kmeans_pred.append(0)

In [ ]:
fowlkes_mallows_score(kmeans_pred, df['toxic']) 

In [ ]:
def accuracy_score_(labels__, labels___):
    score = 0
    for idx, label in enumerate(labels__):
        if label == labels___[idx]:
            score = score + 1

    return score/len(labels___)

In [ ]:
accuracy_score_(kmeans_pred, df['toxic'])

In [ ]:
print(classification_report(df['toxic'], kmeans_pred))

In [ ]:
confusion_matrix(df['toxic'], kmeans_pred)

In [ ]:
ax = sns.heatmap(confusion_matrix(test_labels, prediction), annot = np.array([['6878', '2708'],['3733', '1093']]), cmap=plt.cm.Blues, fmt = '')

### Elbow method

In [ ]:
Y = []
for k in range(1,10):
    kmean_ = KMeans(n_clusters=k).fit(data_)
    Y.append(kmean_.inertia_)

In [ ]:
X = range(1,10)

plt.figure(figsize=(12,6))
plt.plot(X, Y)
plt.plot(2, 7554, 'gD')

plt.text(2.2, 14003, 'Elbow point', bbox=dict(color='green', alpha=0.8))
plt.text(2.38, 13545, 'k = 2', fontsize = 12)

plt.ylabel('Squared distances sum')
plt.xlabel('No of clusters')
plt.title('Elbow Method')

plt.show()

## DBSCAN

In [ ]:
# Dimensionality reduction
SVD = TruncatedSVD(100)
Pca = SVD.fit_transform(data_)

# Makeing the data positive
scaler = MinMaxScaler().fit(Pca)
data_ = scaler.transform(Pca)

In [ ]:
clustering = DBSCAN(eps=0.9, min_samples=2).fit(data_)

In [ ]:
fowlkes_mallows_score(clustering.labels_, data['toxic'])

In [ ]:
accuracy_score_(clustering.labels_, data['toxic'])

###Parameter Tunning

In [ ]:
# Computing a tabel for parameter comparison

Acc = []
Param = []
X_ = []
print('Clusters \t Acc \t eps \t min_samples')
for eps_ in [ 0.6, 0.8, 0.9]:
    for min_samples_ in range(1, 10):
        clustering = DBSCAN(eps=eps_, min_samples=min_samples_).fit(data_)
        n_clusters = len(set(clustering.labels_)) - (1 if -1 in clustering.labels_ else 0)
        print( n_clusters,' \t       ', round(accuracy_score_(clustering.labels_, data['toxic']), 3),'\t ', eps_, '  \t ', min_samples_)
        
        X_.append(accuracy_score_(clustering.labels_, data['toxic']))
        if n_clusters == 2 or n_clusters == 3:
            Acc.append(accuracy_score_(clustering.labels_, data['toxic']))
            Param.append([eps_, min_samples_])

In [ ]:
#print('Best score: ', max(Acc),'\nBest Parameters: eps_ = ', Param[np.argmax(Acc, axis = 0)][0], ', min_sample = ', Param[np.argmax(Acc, axis = 0)][1])#

In [ ]:
Y_ = range(0,27)

plt.figure(figsize=(12,6))
plt.plot(Y_, X_)
plt.plot(22, 0.6602830974188176, 'gD')

plt.text(20.2, 0.635, 'Maximum accuracy', bbox=dict(color='green', alpha=0.8))
plt.text(19.4, 0.615, 'eps_ =  0.9 , min_sample =  1', fontsize = 10)

plt.xlabel('Accuracy')
plt.ylabel('Parameters')
plt.title('Best Parameters DBSCAN')

plt.show()